# Preparativos

In [3]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download zaheenhamidani/ultimate-spotify-tracks-db

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [4]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

urlTrain  = 'https://raw.githubusercontent.com/CyberJuan55/Proyecto-IA/master/DataSet/SpotifyFeatures_train.csv'
urlFull = 'https://raw.githubusercontent.com/CyberJuan55/Proyecto-IA/master/DataSet/SpotifyFeatures.csv'
# el train que nos dieron los profes
#df_train_original= pd.read_csv('/content/SpotifyFeatures_train.csv')
df_train_original = pd.read_csv(urlTrain)
df_train = df_train_original.copy()
# el dataset nuestro
df_full_original = pd.read_csv(urlFull)
df_full = df_full_original.copy()


# Exploracion de datos

In [5]:
df_train.head()

,Unnamed: 0,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,107360,Opera,Richard Wagner,Gotterdammerung (Twilight of the Gods): Prolog...,32pnRCr5u3nzO55MgoEJwV,6,0.89900,0.147,131787,0.016,0.947000,B,0.0889,-24.317,Major,0.0380,137.591,4/4,0.0292
1,165017,Reggaeton,Ronald El Killa,Esta Pena,3uMQ7q5oU0VgYl9SyCF2MR,27,0.00613,0.857,255134,0.599,0.000085,B,0.2090,-5.677,Major,0.0951,96.022,4/4,0.5860
2,141044,Indie,LP,Recovery,1a5P7IvLUhRhLnnbkev3PI,59,0.27400,0.518,234905,0.545,0.000023,A#,0.1830,-7.893,Major,0.0385,75.060,4/4,0.1620
3,141709,Indie,The National,Carin at the Liquor Store,4maI5FwVIzjrErduPjjZLO,53,0.59400,0.540,213733,0.615,0.803000,A#,0.1200,-9.264,Major,0.0340,93.170,3/4,0.2390
4,106321,Opera,Renata Tebaldi,Giordano: Andrea Chenier: Come un bel di di ma...,5VEVnqQu6cFewcbWfbkUeE,8,0.95500,0.363,170240,0.196,0.000115,C#,0.6840,-15.675,Minor,0.0452,63.186,4/4,0.2510


In [6]:
#df_full.head()

In [7]:
df_train.columns

Index(['Unnamed: 0', 'genre', 'artist_name', 'track_name', 'track_id',
       'popularity', 'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence'],
      dtype='object')

In [8]:
df_train.dtypes

Unnamed: 0            int64
genre                object
artist_name          object
track_name           object
track_id             object
popularity            int64
acousticness        float64
danceability        float64
duration_ms           int64
energy              float64
instrumentalness    float64
key                  object
liveness            float64
loudness            float64
mode                 object
speechiness         float64
tempo               float64
time_signature       object
valence             float64
dtype: object

# Limpieza de datos

In [9]:
df_train = df_train.drop(['Unnamed: 0'], axis=1)


In [10]:

indexNames = df_train[ df_train['genre'] == 'Comedy' ].index
df_train.drop(indexNames , inplace=True)

In [11]:
df_train['genre'].value_counts()

Soundtrack          7758
Jazz                7634
Indie               7603
Children’s Music    7554
Pop                 7532
Electronic          7500
Rock                7455
Hip-Hop             7422
Folk                7406
Classical           7395
Rap                 7367
Alternative         7342
Soul                7280
R&B                 7242
World               7211
Blues               7187
Ska                 7154
Reggaeton           7139
Anime               7131
Reggae              7019
Dance               6950
Country             6861
Opera               6612
Movie               6261
Children's Music    4298
A Capella             96
Name: genre, dtype: int64

In [12]:
#columnas con datos faltantes
cols_with_missing = [col for col in df_train.columns if df_train[col].isnull().any()] 
if len(cols_with_missing)==0:
  print('no faltan valores') 

no faltan valores


In [13]:
# saco las filas correspondientes
print(cols_with_missing)
df_train.dropna(axis=0, inplace=True)


[]


# Variables Categoricas

In [14]:
# Variable categoricas
s = (df_train.dtypes == 'object')
object_cols = list(s[s].index)
print(object_cols)

['genre', 'artist_name', 'track_name', 'track_id', 'key', 'mode', 'time_signature']


In [15]:
list_of_timeSignature = df_train['time_signature'].unique()
print(list_of_timeSignature)

['4/4' '3/4' '5/4' '1/4' '0/4']


In [16]:
df_train['time_signature'].value_counts()

4/4    156851
3/4     16544
5/4      3357
1/4      1651
0/4         6
Name: time_signature, dtype: int64

Decidimos no utilizar Time_signature como feature por varios motivos


*   En musica no existe compaces de 1/4 ni de 0/4 entre ambos suman 1656 valores lo que nos parecio un numero considerable
*   En una cancion puede haber multiples cambios de compaces, en la musica progresiva es algo muy comun

*   La mayoria de la musica se encuentra en 4/4 podemos ver que hay 156851 valores que considerablemente mayor a lo suma de todas las otras categorias esto introduceria un gran sesgo en nuestro modelo 










In [17]:
df_train = df_train.drop(['time_signature'], axis=1)

In [18]:
# variables categoricas con baja cardinalidad
categorical_cols = [cname for cname in df_train.columns if
                    df_train[cname].nunique() < 14 and 
                    df_train[cname].dtype == "object"]
print(categorical_cols)

['key', 'mode']


Quedan descartadas las variables categoricas: 

*   Time_signature por los motivos comentados anteriormente
*   track_id: no hay ninguna razon para utilizarla
*   el nombre del artista: son demasiadas variables de momento las sacamos pero es cierto que es un feature interesente 
*   track_name: todos sus valores son distintos 







In [19]:
list_of_modes = df_train['mode'].unique()
print(list_of_modes)

['Major' 'Minor']


In [20]:
list_of_keys = df_train['key'].unique()
print(list_of_keys)


['B' 'A#' 'C#' 'C' 'F#' 'E' 'A' 'D' 'G' 'F' 'G#' 'D#']


# Removiendo datos de columnas faltantes, separacion en entrenamiento y validacion

In [21]:
# Remove rows with missing target, separate target from predictors

X = df_train.drop(columns=['genre'])
y = df_train['genre']


In [22]:
from sklearn.model_selection import train_test_split
X_train_full, X_valid_full, y_train, y_valid = train_test_split(df_train, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

In [23]:
y_train.value_counts().sort_index()

A Capella             75
Alternative         5871
Anime               5734
Blues               5733
Children's Music    3459
Children’s Music    6086
Classical           5914
Country             5447
Dance               5549
Electronic          5993
Folk                5899
Hip-Hop             5917
Indie               6113
Jazz                6098
Movie               5012
Opera               5252
Pop                 6061
R&B                 5807
Rap                 5845
Reggae              5611
Reggaeton           5716
Rock                5971
Ska                 5702
Soul                5853
Soundtrack          6231
World               5778
Name: genre, dtype: int64

In [24]:
y_valid.value_counts().sort_index()

A Capella             21
Alternative         1471
Anime               1397
Blues               1454
Children's Music     839
Children’s Music    1468
Classical           1481
Country             1414
Dance               1401
Electronic          1507
Folk                1507
Hip-Hop             1505
Indie               1490
Jazz                1536
Movie               1249
Opera               1360
Pop                 1471
R&B                 1435
Rap                 1522
Reggae              1408
Reggaeton           1423
Rock                1484
Ska                 1452
Soul                1427
Soundtrack          1527
World               1433
Name: genre, dtype: int64

In [25]:

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

print(numerical_cols)

['popularity', 'acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence']


In [26]:
#Seleccion de columnas
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()


# Un poco de ingenieria de datos

In [27]:
from sklearn.preprocessing import OrdinalEncoder


label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

ordinal_encoder = OrdinalEncoder()
label_X_train[categorical_cols] = ordinal_encoder.fit_transform(X_train[categorical_cols])
label_X_valid[categorical_cols] = ordinal_encoder.transform(X_valid[categorical_cols])




# Probando modelos


###  Regresion Logistica














In [28]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(multi_class = 'multinomial', solver='lbfgs', max_iter=200)

model.fit(label_X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=200,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [29]:
y_pred = model.predict(label_X_valid)

In [51]:
from sklearn.metrics import classification_report, accuracy_score
print(accuracy_score(y_valid, y_pred))

0.17473796311865927


### Random Forest

In [31]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=30, random_state=42)
rfc.fit(label_X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=30,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [50]:
predict = rfc.predict(label_X_valid)
print(accuracy_score(y_valid, predict))


0.3407320217476599


### GridSerchCv en RandonForest

In [33]:
# Numero de arboles
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 80, num = 10)]
# Numero de features considerado al dividir
max_features = ['auto', 'sqrt']
# numero maximo de niveles
max_depth = [2,4]
# Numero minimo de niveles por nodo
min_samples_split = [2, 5]
# Numero minimo de niveles por hoja
min_samples_leaf = [1, 2]
# metodo de seleccion por arbol 
bootstrap = [True, False]

In [34]:
# Grilla
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


In [35]:
from sklearn.model_selection import GridSearchCV
rfc_Grid = GridSearchCV(estimator = rfc, param_grid = param_grid, cv = 3, verbose=2, n_jobs = 4)

In [36]:
rfc_Grid.fit(label_X_train, y_train)


Fitting 3 folds for each of 320 candidates, totalling 960 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:  1.2min
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:  5.8min
[Parallel(n_jobs=4)]: Done 357 tasks      | elapsed: 16.3min
[Parallel(n_jobs=4)]: Done 640 tasks      | elapsed: 30.7min
[Parallel(n_jobs=4)]: Done 960 out of 960 | elapsed: 53.0min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=30, n_jobs=None,
                                              oob_score=False, random_state=42,
                                   

In [37]:
#rfc_Grid.best_params_

{'bootstrap': True,
 'max_depth': 4,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 80}

In [43]:

clf  = RandomForestClassifier(n_estimators=80, random_state=42,bootstrap=True,max_depth= 4, max_features='auto',min_samples_leaf=1,min_samples_split=2)
clf.fit(label_X_train, y_train)
y_clf = clf.predict(label_X_valid)


In [47]:
print(accuracy_score(y_valid, y_clf))


0.3207219326270949


In [39]:
from sklearn.externals import joblib
joblib.dump(rfc_Grid,'modelo_entrenado.pkl')

['modelo_entrenado.pkl']